In [10]:
import requests
from bs4 import BeautifulSoup as soup
import pandas as pd
import uuid
import difflib

In [11]:
player_index = pd.read_csv('../updated_datasets/player_index.csv')
player_index = player_index[player_index.columns[1:]]
player_index

,Player,UUID
0,Kevin Garnett,4035d838-5ed7-4507-86ad-0488a9b9c358
1,Shaquille O'Neal,74b99a51-bb65-436f-8bcc-7022af2a490a
2,Alonzo Mourning,8051eb29-ecd7-46b5-b2fa-415777e11e83
3,Juwan Howard,5bdd7fb5-050d-4b0c-befb-5fbf7109397c
4,Hakeem Olajuwon,be9d9c53-d445-4b8c-acc9-ad1c30e8598b
...,...,...
2224,Jordan Schakel,49852ad9-c5e3-4124-bd75-2ad27b5a859f
2225,Jordan Goodwin,70d9ad35-fb19-497a-a358-d77a2a2e89ba
2226,Craig Sword,c29acdbe-9077-44c0-a8b3-f997ddff72a2
2227,Jaime Echenique,727da240-fbce-4643-ba94-f74509e2e746


In [1]:
table_list = []
df_list = []
def awards_data(website):
    main = soup(requests.get('http://www.espn.com/nba/history/awards').text, 'html.parser')
    links = [a['href'] for a in main.find_all('a', class_='bi', href=True)]
    links.remove('//www.espn.com/nba/history/awards/_/id/34')
    
    for link in links:
        #grab the header of the website page
        name = str(soup(requests.get(f'http:{link}').text).select('h2')[0]).replace(
            '<h2>', '').replace('</h2>', '').replace('<h2>', '').partition('- ')[-1].split()
        
        #'the' doesn't get iterated through so remove it
        if 'the' in name:
            name.remove('the')
            
        #set variable name to the first two capitalized words of the header
        if len(name) > 1:
            for word in name:
                if word[0] != word[0].upper():
                    name.remove(word)
            var_name = (name[0] + '_' + name[1]).lower().replace('-', '_')
        else:
            var_name = ''.join(name).lower()
        
        table_list.append(var_name)
        
        #grab the datset
        globals()[var_name] = pd.read_html(requests.get(f'http:{link}').text)[0]
        
        #clean the data
        globals()[var_name].columns = globals()[var_name].iloc[1].str.capitalize()
        globals()[var_name] = globals()[var_name].tail(-2)

        #some tables have blank year columns so filter for that
        globals()[var_name].fillna(method='ffill', inplace=True)

        #more cleaning
        globals()[var_name] = globals()[var_name][globals()[var_name]['Year'].astype('int') >= 2000]
        globals()[var_name] = globals()[var_name][globals()[var_name].columns[:4]]

        #gather uuids
        globals()[var_name] = pd.merge(left=globals()[var_name], right=player_index, how='left', on='Player')
        
        #locate mispelled names
        null_players = []
        for row in globals()[var_name][globals()[var_name]['UUID'].isnull()]['Player']:
            null_players.append(row)
        null_players = pd.DataFrame(null_players, columns=['Player']).drop_duplicates()
        
        #locate correct names from index
        nearest_name = []
        original_name = []
        for name in null_players['Player']:
            close_matches = difflib.get_close_matches(name, player_index['Player'], 1)
            for x in close_matches:
                    nearest_name.append(x)
                    original_name.append(name)
        nearest_name = pd.DataFrame(nearest_name, columns=['New_name'])
        nearest_name['Original_name'] = original_name
        
        #fix mispelled names 
        for bad_name in globals()[var_name]['Player']:
            if bad_name in [x for x in nearest_name['Original_name']]:
                globals()[var_name].loc[globals()[var_name][globals()[var_name]['Player'] == bad_name].index, 'Player'] = [
                    x for x in nearest_name[nearest_name['Original_name'] == bad_name]['New_name']][0]
        
        #merge w/ index table to add unique ids and save csvs
        globals()[var_name] = pd.merge(left=globals()[var_name], right=player_index, how='left', on='Player')
        globals()[var_name].drop(columns=['UUID_x'], inplace=True)
        globals()[var_name].rename(columns={'UUID_y':'UUID'}, inplace=True)
        globals()[var_name].to_csv(f'../updated_datasets/awards_data/{var_name}.csv')
        
        #add dataframe to my output list of dataframes
        df_list.append(globals()[var_name])

    return table_list

In [13]:
awards_data('http://www.espn.com/nba/history/awards')

['mvp',
 'defensive_player',
 'rookie_year',
 'sixth_man',
 'most_improved',
 'finals_mvp',
 'all_star_mvp',
 'all_nba_1st',
 'all_nba_2nd',
 'all_nba_3rd',
 'all_rookie_1st',
 'all_rookie_2nd',
 'all_defensive_1st',
 'all_defensive_2nd']

In [14]:
table_list

['mvp',
 'defensive_player',
 'rookie_year',
 'sixth_man',
 'most_improved',
 'finals_mvp',
 'all_star_mvp',
 'all_nba_1st',
 'all_nba_2nd',
 'all_nba_3rd',
 'all_rookie_1st',
 'all_rookie_2nd',
 'all_defensive_1st',
 'all_defensive_2nd']

In [58]:
sixth_man.head()

,Year,Player,Pos,Team,UUID
0,2022,Tyler Herro,PG,Miami Heat,b1359ec1-999b-418f-84e8-791659e3d59c
1,2021,Jordan Clarkson,PG,Utah Jazz,d7aa0308-767f-4f9a-905e-8800e7c41ef7
2,2020,Montrezl Harrell,F,LA Clippers,64379148-f232-45b7-abe0-c3f3b5c6138a
3,2019,Louis Williams,G,LA Clippers,d861edd2-969c-4d79-ae0d-9c6d1ffa11d2
4,2018,Louis Williams,G,LA Clippers,d861edd2-969c-4d79-ae0d-9c6d1ffa11d2


In [57]:
mvp.head()

,Year,Player,Pos,Team,UUID
0,2022,Nikola Jokic,C,Denver Nuggets,4d321b07-ade4-497c-be07-e8335e03b288
1,2021,Nikola Jokic,C,Denver Nuggets,4d321b07-ade4-497c-be07-e8335e03b288
2,2020,Giannis Antetokounmpo,F,Milwaukee Bucks,f5e976f0-2383-4f34-869e-912b0daebc1d
3,2019,Giannis Antetokounmpo,F,Milwaukee Bucks,f5e976f0-2383-4f34-869e-912b0daebc1d
4,2018,James Harden,G,Houston Rockets,ef0995d8-4803-48be-b6b8-d6cdc7e8c433


In [35]:
all_rookie_1st.loc[all_rookie_1st[all_rookie_1st['Player'] == 'Nene Hilario'].index, 'Player'] = 'Nenê'

In [40]:
all_rookie_1st.loc[all_rookie_1st[all_rookie_1st['Player'] == 'Nenê'].index, 'UUID'] = '71b3fc68-5b92-4644-af01-caa989d432ad'

In [36]:
def readd_uuids(table):
    table = pd.merge(left=table, right=player_index, how='left', on='Player')
    table.drop(columns=['UUID_x'], inplace=True)
    table.rename(columns={'UUID_y':'UUID'}, inplace=True)
    return table

In [37]:
#readd_uuids(all_rookie_1st)

,Year,Player,Pos,Team,UUID
0,2022,Evan Mobley,C,Cleveland Cavaliers,fe25f98b-f0ba-4665-96b1-b7ad652a993a
1,2022,Cade Cunningham,PG,Detroit Pistons,2087fa55-fdb9-4e68-9004-1999c4ea8e5d
2,2022,Scottie Barnes,SF,Toronto Raptors,b6067532-6a32-4682-b8f1-4c88f3a771b2
3,2022,Jalen Green,SG,Houston Rockets,47a1a42d-9ae5-4215-a25b-8afe1061e4d6
4,2022,Franz Wagner,SF,Orlando Magic,a29bc85d-d0e0-459e-835e-6a92914d00d4
...,...,...,...,...,...
113,2000,Elton Brand,PF,Chicago Bulls,d63e5ac8-f1ef-4bd5-8d76-8f4d65432d8a
114,2000,Steve Francis,PG,Houston Rockets,e80d5f18-0228-43d7-89ad-76d953c0fc4f
115,2000,Andre Miller,PG,Cleveland Cavaliers,c572c106-7e60-4ff3-a115-1f32869d30c9
116,2000,Lamar Odom,PF,Los Angeles Clippers,2c755913-c6d6-417f-9fcc-e902381f5aaf


In [41]:
all_rookie_1st[all_rookie_1st['Player'] == 'Nenê']

,Year,Player,Pos,Team,UUID
100,2003,Nenê,C,Denver Nuggets,71b3fc68-5b92-4644-af01-caa989d432ad


In [42]:
def define_nulls(tables):
    null_players = []
    for table in tables:
        for row in table[table['UUID'].isnull()]['Player']:
            null_players.append(row)
    null_players = pd.DataFrame(null_players, columns=['Player']).drop_duplicates()
    return null_players

In [45]:
define_nulls(df_list)

,Player


In [46]:
def search_index(table):
    return sum([True if len(cell) > 0 else False for cell in [player_index[player_index['Player'] == row] for row in table['Player']]])

In [47]:
search_index(null_players)

0

In [49]:
def player_lookup(players, table):
    player_lookup = pd.DataFrame(columns = ['Player', 'UUID'])
    player_name = []
    player_uuid = []
    for player in players:
            for name in table['Player']:
                if player.lower() in name.lower():
                    player_name.append(name)
                    for uuid in table[table['Player'] == name]['UUID']:
                        player_uuid.append(uuid)
    player_lookup['Player'] = player_name
    player_lookup['UUID'] = player_uuid
    return player_lookup

In [50]:
def nearest_name(players):
    nearest_name = []
    original_name = []
    for name in players:
        close_matches = difflib.get_close_matches(name, player_index['Player'], 1)
        for x in close_matches:
                nearest_name.append(x)
                original_name.append(name)
    nearest_name = pd.DataFrame(nearest_name, columns=['New_name'])
    nearest_name['Original_name'] = original_name
    return nearest_name

In [51]:
nearest_name(null_players['Player'])

,New_name,Original_name


In [52]:
table_list

['mvp',
 'defensive_player',
 'rookie_year',
 'sixth_man',
 'most_improved',
 'finals_mvp',
 'all_star_mvp',
 'all_nba_1st',
 'all_nba_2nd',
 'all_nba_3rd',
 'all_rookie_1st',
 'all_rookie_2nd',
 'all_defensive_1st',
 'all_defensive_2nd']

In [53]:
def patch_nulls(tables):
    names = nearest_name(define_nulls(tables)['Player'])    
    names['Check'] = ''
    for table in tables:
        for bad_name in table['Player']:
            if bad_name in [x for x in names['Original_name']]:
                table.loc[table[table['Player'] == bad_name].index, 'Player'] = [
                    x for x in names[names['Original_name'] == bad_name]['New_name']][0]
                names['Check'] = True
        table = pd.merge(left=table, right=player_index, how='left', on='Player')
    return names

In [19]:
#patch_nulls(df_list)

,New_name,Original_name,Check
0,Amare Stoudemire,Amar'e Stoudemire,True
1,Louis Williams,Lou Williams,True
2,Hidayet Turkoglu,Hedo Turkoglu,True
3,Predrag Stojakovic,Peja Stojakovic,True
4,Marvin Bagley,Marvin Bagley III,True
5,Jaren Jackson Jr,Jaren Jackson Jr.,True
6,Tim Hardaway Jr,Tim Hardaway Jr.,True
7,OJ Mayo,O.J. Mayo,True
8,Nenê,Nenê,True
9,Herb Jones,Herbert Jones,True


In [54]:
player_lookup(['Nen'], player_index)

,Player,UUID
0,Nenê,71b3fc68-5b92-4644-af01-caa989d432ad
1,Nenad Krstic,a9c69c50-2092-4132-a581-4f6deeabe74b
2,Lauri Markkanen,14de75ea-1ce3-46a6-8393-bc7562ad3f0b


In [55]:
all_rookie_1st[all_rookie_1st['Player'] == 'Nenê']

,Year,Player,Pos,Team,UUID
100,2003,Nenê,C,Denver Nuggets,71b3fc68-5b92-4644-af01-caa989d432ad


In [56]:
player_index[player_index['Player'] == 'Nenê']

,Player,UUID
537,Nenê,71b3fc68-5b92-4644-af01-caa989d432ad
